# A Test Notebook to Draft Adversarial attack experiments

In [4]:
from PIL import Image
import torch
import torch.nn as nn
import tensorflow as tf
import numpy as np
import pandas as pd

In [5]:
import timm
# Load pretrained PyTorch feature extractor
model = timm.create_model(
    'resnet18.a1_in1k',
    pretrained=True,
    num_classes=0,  # remove classifier nn.Linear
)
feature_extractor = model.eval()

# get model specific transforms (normalization, resize)
data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)

In [6]:
import pickle

# Load test data from pickle file
#with open('W:/DS/Project/CNN Prototype/cifar-10-batches-py/test_batch', 'rb') as f:  # Replace with actual path
with open('E:/Work/DS/Project/CNN Protoype/cifar-10-batches-py/test_batch', 'rb') as f:
    data = pickle.load(f, encoding='bytes')

BATCH_NUMBER = 10

def load_batch(data, BATCH_NUMBER = BATCH_NUMBER, iteration = 0):
    # Handle different data formats
    if isinstance(data, dict):
        x_test = data[b'data'][iteration*BATCH_NUMBER:(iteration + 1)*BATCH_NUMBER]
        x_test = [Image.fromarray(image.reshape(3,32,32).transpose(1,2,0)) for image in x_test]
        y_test = data[b'labels'][iteration*BATCH_NUMBER:(iteration + 1)*BATCH_NUMBER]
    else:
        x_test, y_test = data

    y_test = pd.get_dummies(y_test).values

    # Convert to PyTorch tensors for the feature extractor
    x_test_pt = torch.stack([transforms(image) for image in x_test])  # NHWC to NCHW
    y_test_pt = torch.tensor(y_test).float()
    return x_test_pt, y_test_pt


## Attack types

### IFGSM

In [7]:
# Iterative FGSM (IFGSM) Attack Function
def ifgsm_attack(input_image, label, epsilon, alpha = 1, iterations = 5, classifier = None):
    adv_image = input_image.clone().detach()
    for _ in range(iterations):
        adv_image.requires_grad = True
        features = feature_extractor(adv_image)
        features_np = features.detach().cpu().numpy()

        with tf.GradientTape() as tape:
            features_tf = tf.convert_to_tensor(features_np)
            tape.watch(features_tf)
            predictions = classifier(features_tf)
            loss = tf.keras.losses.categorical_crossentropy(label, predictions)

        grad_features = tape.gradient(loss, features_tf).numpy()
        grad_features_pt = torch.tensor(grad_features).to(adv_image.device)
        features.backward(grad_features_pt)
        grad_input = adv_image.grad.data

        adv_image = adv_image + alpha * grad_input.sign()
        perturbation = torch.clamp(adv_image - input_image, min=-epsilon, max=epsilon)
        adv_image = (input_image + perturbation).detach()

    return adv_image

### IFGM

In [8]:
def ifgm_attack(input_image, label, epsilon, alpha = 1, iterations = 5, decay_factor=1.0, classifier = None):
    adv_image = input_image.clone().detach()
    momentum = torch.zeros_like(adv_image)

    for _ in range(iterations):
        adv_image.requires_grad = True
        features = feature_extractor(adv_image)
        features_np = features.detach().cpu().numpy()

        with tf.GradientTape() as tape:
            features_tf = tf.convert_to_tensor(features_np)
            tape.watch(features_tf)
            predictions = classifier(features_tf)
            loss = tf.keras.losses.categorical_crossentropy(label, predictions)

        grad_features = tape.gradient(loss, features_tf).numpy()
        grad_features_pt = torch.tensor(grad_features).to(adv_image.device)
        features.backward(grad_features_pt)
        grad_input = adv_image.grad.data

        momentum = decay_factor * momentum + grad_input / torch.norm(grad_input, p=1)
        adv_image = adv_image + alpha * momentum
        perturbation = torch.clamp(adv_image - input_image, min=-epsilon, max=epsilon)
        adv_image = (input_image + perturbation).detach()

    return adv_image

### L-BFGS Attack Function

In [9]:
def lbfgs_attack(input_image, label, classifier = None):
    def loss_fn(perturbation):
        pert_image = input_image + torch.tensor(perturbation, dtype=torch.float32).reshape_as(input_image)
        features = feature_extractor(pert_image)
        predictions = classifier(features.detach().cpu().numpy())
        loss = tf.keras.losses.categorical_crossentropy(label, predictions)
        return loss.numpy().astype(np.float64)

    perturbation = np.zeros_like(input_image.cpu().numpy())
    result = minimize(loss_fn, perturbation, method='L-BFGS-B')
    adversarial_image = input_image + torch.tensor(result.x, dtype=torch.float32).reshape_as(input_image)

    return adversarial_image

### DeepFool

In [10]:
def deepfool_attack(input_image, label, max_iter=5, classifier = None):
    adv_image = input_image.clone().detach()
    for _ in range(max_iter):
        adv_image.requires_grad = True
        features = feature_extractor(adv_image)
        features_np = features.detach().cpu().numpy()
        predictions = classifier(features_np)

        if np.argmax(predictions) != np.argmax(label):
            break

        with tf.GradientTape() as tape:
            features_tf = tf.convert_to_tensor(features_np)
            tape.watch(features_tf)
            loss = tf.keras.losses.categorical_crossentropy(label, predictions)

        grad_features = tape.gradient(loss, features_tf).numpy()
        grad_features_pt = torch.tensor(grad_features).to(adv_image.device)
        features.backward(grad_features_pt)
        grad_input = adv_image.grad.data

        perturbation = grad_input / torch.norm(grad_input)
        adv_image = adv_image + perturbation.sign() * 0.01  # small step size

    return adv_image

# The experiment

In [11]:

# Evaluate on Adversarial Examples
def evaluate_adversarial(epsilon=0.01, classifiers = None, num_samples=BATCH_NUMBER, classifiers_names = None, attacks_names = None):
    correct = np.zeros(len(classifiers_names))
    adv_correct = np.zeros((len(attacks_names),len(classifiers_names)))
    for i in range(1):
        x_test_pt, y_test_pt = load_batch(data, BATCH_NUMBER = BATCH_NUMBER, iteration = i)
        for j in range(num_samples):
            image = x_test_pt[j:j+1]
            label = y_test_pt[j:j+1]
            # Forward pass through both models
            features = feature_extractor(image).detach().cpu().numpy()
            for k,classifier in enumerate(classifiers):
                pred = classifier.predict(features)
                if np.argmax(pred) == np.argmax(label):
                    correct[k] += 1
                for n, attack_name in enumerate(attacks_names):
                    if (attack_name == 'FGM'):
                        adv_image = ifgm_attack(image.clone(), label, epsilon, iterations = 1, decay_factor=1.0, classifier = classifier)
                    elif  (attack_name == 'IFGM'):
                        adv_image = ifgm_attack(image.clone(), label, epsilon, iterations = 5, decay_factor=1.0, classifier = classifier)
                    elif  (attack_name == 'FGSM'):
                        adv_image = ifgsm_attack(image.clone(), label, epsilon, iterations = 1, decay_factor=1.0, classifier = classifier)
                    elif  (attack_name == 'IFGSM'):
                        adv_image = ifgsm_attack(image.clone(), label, epsilon, iterations = 5, decay_factor=1.0, classifier = classifier)
                    elif  (attack_name == 'L-BFGS'):
                        adv_image = lbfgs_attack(image.clone(), label, classifier = classifier)
                    elif  (attack_name == 'deepfool'):
                        adv_image = deepfool_attack(image.clone(), label, max_iter=5, classifier = classifier)


                    adv_features = feature_extractor(adv_image).detach().cpu().numpy()
                    adv_pred = classifier.predict(adv_features)
                    if np.argmax(adv_pred) == np.argmax(label):
                        adv_correct[n,k] += 1
    accuracy = correct / 10
    adv_accuracy = adv_correct / 10
    for k,classifier in enumerate(classifiers):
        print('Baseline Accuracy for ' + classifiers_names[k] + f' (ε = {epsilon}): {accuracy[k] * 100:.2f}%')
        for n, attacks_name in enumerate(attacks_names):
            print('Adversarial Accuracy for ' + classifiers_names[k] + 'on' + attacks_name + f' (ε = {epsilon}): {adv_accuracy[n,k] * 100:.2f}%')
    return accuracy, adv_accuracy

# The tests

In [15]:
from custom_RBF_neuron import RBFLayer
from custom_ENN_layerV2 import ENNLayer

In [16]:
# Load TensorFlow classifier
neurons = ['MP','RBF','ECF']
MP_classifier = tf.keras.models.load_model('E:/Work/DS/Project/CNN Experiment/ResNet18/CIFAR10/CIFAR10_ResNet18_MP_epochs_20.keras')  # Replace with actual path
RBF_classifier = tf.keras.models.load_model('E:/Work/DS/Project/CNN Experiment/ResNet18/CIFAR10/CIFAR10_ResNet18_RBF_epochs_20.keras', custom_objects={'RBFLayer': RBFLayer})  # Replace with actual path
ECF_classifier = tf.keras.models.load_model('E:/Work/DS/Project/CNN Experiment/ResNet18/CIFAR10/CIFAR10_ResNet18_ENN_epochs_20.keras', custom_objects={'ENNLayer': ENNLayer})  # Replace with actual path

'''
MP_classifier = tf.keras.models.load_model('W:/DS/Project/CNN Experiment/ResNet18/CIFAR10/CIFAR10_ResNet18_MP_epochs_20.keras')  # Replace with actual path
RBF_classifier = tf.keras.models.load_model('W:/DS/Project/CNN Experiment/ResNet18/CIFAR10/CIFAR10_ResNet18_RBF_epochs_20.keras', custom_objects={'RBFLayer': RBFLayer})  # Replace with actual path
ECF_classifier = tf.keras.models.load_model('W:/DS/Project/CNN Experiment/ResNet18/CIFAR10/CIFAR10_ResNet18_ENN_epochs_20.keras', custom_objects={'ENNLayer': ENNLayer})  # Replace with actual path
'''

classifiers = [MP_classifier,RBF_classifier,ECF_classifier]

# setup attacks
attack_types_name = ['FGM', 'IFGM', 'FSGM', 'IFGSM', 'L-BFGS', 'Deepfool']
# Run evaluation
evaluate_adversarial(epsilon=0.007, classifiers = classifiers, classifiers_names = neurons, attacks_names = attack_types_name)


TypeError: <class 'keras.src.models.functional.Functional'> could not be deserialized properly. Please ensure that components that are Python object instances (layers, models, etc.) returned by `get_config()` are explicitly deserialized in the model's `from_config()` method.

config={'module': 'keras.src.models.functional', 'class_name': 'Functional', 'config': {'name': 'functional', 'trainable': True, 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_shape': [None, 512], 'dtype': 'float32', 'sparse': False, 'name': 'input_layer'}, 'registered_name': None, 'name': 'input_layer', 'inbound_nodes': []}, {'module': 'custom_RBF_neuron', 'class_name': 'RBFLayer', 'config': {'name': 'rbf_layer', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'units': 100, 'gamma': {'class_name': '__tensor__', 'config': {'value': 602.0853881835938, 'dtype': 'float32'}}}, 'registered_name': '